## Gathering Dependencies

_Importing Required Libraries_

In [1]:
pip install hampel

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Anurag Dutta\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import keras
import tensorflow as tf
from hampel import hampel
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
from numpy import array

## Pretraining

_Lotka Volterra Intermittency_

The `lotka_volterra_intermittency.dat` is generated from the Runge Kutta Method.

This feeds the model with the dynamics of the Lotka Volterra Equations

In [3]:
data = np.genfromtxt('datasets/lotka_volterra_intermittency.dat')
training_set = pd.DataFrame(data).reset_index(drop=True)
training_set = training_set.iloc[:,1]

## Computing the Gradients

_Calculating the value of_ $\frac{dx}{dt}$, _and_ $\frac{d^2x}{dt^2}$

In [4]:
t_diff = 1 # Daily Data
print(training_set.max())
gradient_t = (training_set.diff()/t_diff).iloc[1:]
print(gradient_t)
gradient_tt = (gradient_t.diff()/t_diff).iloc[1:]
print(gradient_tt)

215.48740705
1        -1.078114e-02
2        -1.066358e-02
3        -1.054729e-02
4        -1.043229e-02
5        -1.031852e-02
              ...     
139867   -1.113760e-34
139868   -1.101721e-34
139869   -1.089814e-34
139870   -1.078034e-34
139871   -1.066382e-34
Name: 1, Length: 139871, dtype: float64
2         1.175600e-04
3         1.162900e-04
4         1.150000e-04
5         1.137700e-04
6         1.125200e-04
              ...     
139867    1.217000e-36
139868    1.203900e-36
139869    1.190740e-36
139870    1.177950e-36
139871    1.165160e-36
Name: 1, Length: 139870, dtype: float64


## Loading Datasets

_TURKEY EARTHQUAKE_

In [5]:
data = pd.read_csv("datasets/eq.csv")
training_set = data.iloc[:, 4]
print(training_set)

0       4.4
1       5.2
2       4.8
3       4.4
4       5.7
       ... 
6569    4.5
6570    4.0
6571    4.4
6572    4.1
6573    4.0
Name: Magnitude, Length: 6574, dtype: float64


In [6]:
training_set = training_set.reset_index(drop=True)
gradient_t = gradient_t.reset_index(drop=True)
gradient_tt = gradient_tt.reset_index(drop=True)
print(gradient_t)
print(gradient_tt)

0        -1.078114e-02
1        -1.066358e-02
2        -1.054729e-02
3        -1.043229e-02
4        -1.031852e-02
              ...     
139866   -1.113760e-34
139867   -1.101721e-34
139868   -1.089814e-34
139869   -1.078034e-34
139870   -1.066382e-34
Name: 1, Length: 139871, dtype: float64
0         1.175600e-04
1         1.162900e-04
2         1.150000e-04
3         1.137700e-04
4         1.125200e-04
              ...     
139865    1.217000e-36
139866    1.203900e-36
139867    1.190740e-36
139868    1.177950e-36
139869    1.165160e-36
Name: 1, Length: 139870, dtype: float64


In [7]:
print(gradient_t.shape)
print(training_set.shape[:-1])
df = pd.concat((training_set[:-1], gradient_t), axis=1)
gradient_tt.columns = ["grad_tt"]
df = pd.concat((df[:-1], gradient_tt), axis=1)
df.columns = ['y_t', 'grad_t', 'grad_tt']

(139871,)
()


## Preprocessing the data into supervised learning

In [8]:
def Supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
      cols.append(df.shift(-i))
      if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
      else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
       agg.dropna(inplace=True)
    return agg    

In [9]:
dat = Supervised(df.values, n_in = 10, n_out = 365)
dat.columns

Index(['var1(t-10)', 'var2(t-10)', 'var3(t-10)', 'var1(t-9)', 'var2(t-9)',
       'var3(t-9)', 'var1(t-8)', 'var2(t-8)', 'var3(t-8)', 'var1(t-7)',
       ...
       'var3(t+361)', 'var1(t+362)', 'var2(t+362)', 'var3(t+362)',
       'var1(t+363)', 'var2(t+363)', 'var3(t+363)', 'var1(t+364)',
       'var2(t+364)', 'var3(t+364)'],
      dtype='object', length=1125)

In [10]:
data = Supervised(df.values, n_in = 10, n_out = 365)
data.drop(['var2(t-10)', 'var3(t-10)', 'var2(t-9)', 'var3(t-9)', 'var2(t-8)',
       'var3(t-8)', 'var2(t-7)', 'var3(t-7)', 'var2(t-6)', 'var3(t-6)',
       'var2(t-5)', 'var3(t-5)', 'var2(t-4)', 'var3(t-4)', 'var2(t-2)',
       'var3(t-2)', 'var2(t-1)', 'var3(t-1)','var2(t-3)', 'var3(t-3)'], axis = 1, inplace = True)#,18,19
print(data.head())
print(data.columns)

    var1(t-10)  var1(t-9)  var1(t-8)  var1(t-7)  var1(t-6)  var1(t-5)  \
10         4.4        5.2        4.8        4.4        5.7        4.3   
11         5.2        4.8        4.4        5.7        4.3        4.0   
12         4.8        4.4        5.7        4.3        4.0        4.8   
13         4.4        5.7        4.3        4.0        4.8        5.2   
14         5.7        4.3        4.0        4.8        5.2        5.1   

    var1(t-4)  var1(t-3)  var1(t-2)  var1(t-1)  ...  var3(t+361)  var1(t+362)  \
10        4.0        4.8        5.2        5.1  ...     0.000002          5.7   
11        4.8        5.2        5.1        4.7  ...     0.000002          4.9   
12        5.2        5.1        4.7        4.6  ...     0.000002          5.7   
13        5.1        4.7        4.6        4.4  ...     0.000002          5.4   
14        4.7        4.6        4.4        4.7  ...     0.000002          5.2   

    var2(t+362)  var3(t+362)  var1(t+363)  var2(t+363)  var3(t+363)  \
10 

## Train and Test Split

In [11]:
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train_1 = np.array(data[0:train_size])
test_1 = np.array(data[train_size:len(data)])
scaler = MinMaxScaler(feature_range=(0, 1))
train = scaler.fit_transform(train_1)
test = scaler.transform(test_1)
trainY = train[:,-9:]
trainX = train[:,:-9]
testY = test[:,-9:]
testX = test[:,:-9]
trainX = trainX.reshape((trainX.shape[0], 1, trainX.shape[1]))
testX = testX.reshape((testX.shape[0], 1, testX.shape[1]))
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

(4959, 1, 1096) (4959, 9) (1240, 1, 1096) (1240, 9)


## Model

In [12]:
model = Sequential()
model.add(LSTM(50, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(9))
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(trainX, trainY, epochs=500, batch_size=64, validation_data=(testX, testY), shuffle=False)

Epoch 1/500
78/78 [==============================] - 3s 10ms/step - loss: 0.0178 - val_loss: 0.2576
Epoch 2/500
78/78 [==============================] - 0s 4ms/step - loss: 0.0110 - val_loss: 0.2571
Epoch 3/500
78/78 [==============================] - 0s 5ms/step - loss: 0.0085 - val_loss: 0.2585
Epoch 4/500
78/78 [==============================] - 0s 4ms/step - loss: 0.0069 - val_loss: 0.2562
Epoch 5/500
78/78 [==============================] - 0s 4ms/step - loss: 0.0058 - val_loss: 0.2571
Epoch 6/500
78/78 [==============================] - 0s 4ms/step - loss: 0.0066 - val_loss: 0.2590
Epoch 7/500
78/78 [==============================] - 0s 5ms/step - loss: 0.0057 - val_loss: 0.2573
Epoch 8/500
78/78 [==============================] - 0s 5ms/step - loss: 0.0055 - val_loss: 0.2580
Epoch 9/500
78/78 [==============================] - 1s 7ms/step - loss: 0.0057 - val_loss: 0.2567
Epoch 10/500
78/78 [==============================] - 0s 5ms/step - loss: 0.0058 - val_loss: 0.2565
Epoch 11

## Metrics

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
yhat = model.predict(testX)
print(yhat.shape)
testX = testX.reshape((testX.shape[0], testX.shape[2]))
print(testX.shape)
inv_yhat = np.concatenate((testX, yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat1 = inv_yhat[:, -3:]
inv_yhat = inv_yhat[:, -3]
inv_y = np.concatenate((testX, testY), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y1 = inv_y[:, -3:]
inv_y = inv_y[:, -3]
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

39/39 [==============================] - 1s 2ms/step
(1240, 9)
(1240, 1096)
Test RMSE: 0.424
Test MAE: 0.286
